In [1]:
pip install pyspark==3.2.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.5/281.5 MB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 13.8 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.2.3-py2.py3-none-any.whl size=281990661 sha256=139080b21b62bf2ac959ef1da34f76f3f279028a5ad6dc52340f0cd4c24f2dd1
  Stored in directory: /root/.cache/pip/wheels/a5/47/e2/6d348c272c154dec299a9217d4772cae3fc68777a6c495347f
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
      .master("local[1]") \
      .appName("PySpark Demo") \
      .getOrCreate()


In [4]:
##Load data and create spark data frame

In [5]:
rdd = spark.sparkContext.textFile("/content/Capstone market analysis (4).csv")

In [6]:
df = spark.read.csv("/content/Capstone market analysis (4).csv",header = True)
df.printSchema()


root
 |-- age: string (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: string (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: string (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- campaign: string (nullable = true)
 |-- pdays: string (nullable = true)
 |-- previous: string (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- y: string (nullable = true)



In [7]:
df.head()

Row(age='58', job='management', marital='married', education='tertiary', default='no', balance='2143', housing='yes', loan='no', contact='unknown', day='5', month='may', duration='261', campaign='1', pdays='-1', previous='0', poutcome='unknown', y='no')

In [8]:
df.show()

+---+------------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+
|age|         job| marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|  y|
+---+------------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+
| 58|  management| married| tertiary|     no|   2143|    yes|  no|unknown|  5|  may|     261|       1|   -1|       0| unknown| no|
| 44|  technician|  single|secondary|     no|     29|    yes|  no|unknown|  5|  may|     151|       1|   -1|       0| unknown| no|
| 33|entrepreneur| married|secondary|     no|      2|    yes| yes|unknown|  5|  may|      76|       1|   -1|       0| unknown| no|
| 47| blue-collar| married|  unknown|     no|   1506|    yes|  no|unknown|  5|  may|      92|       1|   -1|       0| unknown| no|
| 33|     unknown|  single|  unknown|     no|      1|     no|  no|unknown|  5|  may

In [9]:
df.createOrReplaceTempView("bank")
df.printSchema()

root
 |-- age: string (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: string (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: string (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- campaign: string (nullable = true)
 |-- pdays: string (nullable = true)
 |-- previous: string (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- y: string (nullable = true)



In [10]:
## Marketing Success rate

In [11]:
df1 = spark.sql("select count(*) from bank")
df1.show()

+--------+
|count(1)|
+--------+
|   45211|
+--------+



In [12]:
df2 = spark.sql("select count(*) as subscribed from bank where y = 'yes'")
df2.show()
df14 = spark.sql("select count(*) as total from bank")
df14.show()

+----------+
|subscribed|
+----------+
|      5289|
+----------+

+-----+
|total|
+-----+
|45211|
+-----+



In [13]:
df2.createOrReplaceTempView("a")

In [14]:
df14.createOrReplaceTempView("b")

In [15]:
df_final = spark.sql("select (s.subscribed/t.total)*100 as success_percent from a s join b t ")
df_final.show()

+------------------+
|   success_percent|
+------------------+
|11.698480458295547|
+------------------+



In [16]:
df_a = spark.sql("select count(*) as not_subscribed from bank where y='no'")
df_a.show()
df_a.createOrReplaceTempView("not")

+--------------+
|not_subscribed|
+--------------+
|         39922|
+--------------+



In [17]:
df_b = spark.sql("select count(*) as totally from bank")
df_b.show()
df_b.createOrReplaceTempView("tot")

+-------+
|totally|
+-------+
|  45211|
+-------+



In [18]:
##Marketing Failure Rate

In [19]:
df_finals = spark.sql("select (s.not_subscribed/t.totally)*100 as failure_percent from not s join tot t ")
df_finals.show()

+-----------------+
|  failure_percent|
+-----------------+
|88.30151954170445|
+-----------------+



In [20]:
df3=spark.sql("select y, count(y) from bank group by y")
df3.show()

+---+--------+
|  y|count(y)|
+---+--------+
| no|   39922|
|yes|    5289|
+---+--------+



In [21]:
##Maximum, Mean and Minimum age of average target customer

In [22]:
df5 = spark.sql("select max(age) from bank")
df5.show()

+--------+
|max(age)|
+--------+
|      95|
+--------+



In [23]:
df6 = spark.sql("select min(age) from bank")
df6.show()

+--------+
|min(age)|
+--------+
|      18|
+--------+



In [24]:
df7 = spark.sql("select avg(age) from bank")
df7.show()

+-----------------+
|         avg(age)|
+-----------------+
|40.93621021432837|
+-----------------+



In [25]:
## average balance

In [26]:
df8 = spark.sql("select avg(balance) from bank")
df8.show()

+------------------+
|      avg(balance)|
+------------------+
|1362.2720576850766|
+------------------+



In [27]:
##Median balance

In [28]:
df9 = spark.sql("SELECT percentile_approx(balance, 0.5) FROM bank")
df9.show()

+--------------------------------------+
|percentile_approx(balance, 0.5, 10000)|
+--------------------------------------+
|                                 448.0|
+--------------------------------------+



In [29]:
##Check if age matters in marketing subscription for deposit

In [30]:
df10 = spark.sql("select age, count(*) as number from bank where y='yes' group by age order by number desc")
df10.show()

+---+------+
|age|number|
+---+------+
| 32|   221|
| 30|   217|
| 33|   210|
| 35|   209|
| 31|   206|
| 34|   198|
| 36|   195|
| 29|   171|
| 37|   170|
| 28|   162|
| 38|   144|
| 39|   143|
| 27|   141|
| 26|   134|
| 41|   120|
| 46|   118|
| 40|   116|
| 47|   113|
| 25|   113|
| 42|   111|
+---+------+
only showing top 20 rows



In [31]:
 ##Check if marital status mattered for subscription to deposit.

In [32]:
df12 = spark.sql("select marital, count(*) as number from bank where y='yes' group by marital order by number desc")
df12.show()

+--------+------+
| marital|number|
+--------+------+
| married|  2755|
|  single|  1912|
|divorced|   622|
+--------+------+



In [33]:
## Check if age and marital status together mattered for subscription to deposit scheme

In [34]:
df13 = spark.sql("select age, marital, count(*) as number from bank where y='yes' group by age,marital order by number desc")
df13.show()

+---+-------+------+
|age|marital|number|
+---+-------+------+
| 30| single|   151|
| 28| single|   138|
| 29| single|   133|
| 32| single|   124|
| 26| single|   121|
| 34|married|   118|
| 31| single|   111|
| 27| single|   110|
| 35|married|   101|
| 36|married|   100|
| 25| single|    99|
| 37|married|    98|
| 33|married|    97|
| 33| single|    97|
| 32|married|    87|
| 39|married|    87|
| 38|married|    86|
| 35| single|    84|
| 47|married|    83|
| 46|married|    80|
+---+-------+------+
only showing top 20 rows



In [36]:
 ##Do feature engineering for column—age and find right age effect on campaign

In [35]:
from pyspark.sql.functions import lit
from pyspark.sql.functions import when
df.withColumn("agedf", \
   when((df.age < 20), lit("Teen")) \
     .when((df.age > 20) & (df.age <= 32), lit("Young")) \
     .when((df.age > 33) & (df.age <= 55), lit("Middle Aged")) \
     .otherwise(lit("Old")) \
  ).show()

+---+------------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+-----------+
|age|         job| marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|  y|      agedf|
+---+------------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+-----------+
| 58|  management| married| tertiary|     no|   2143|    yes|  no|unknown|  5|  may|     261|       1|   -1|       0| unknown| no|        Old|
| 44|  technician|  single|secondary|     no|     29|    yes|  no|unknown|  5|  may|     151|       1|   -1|       0| unknown| no|Middle Aged|
| 33|entrepreneur| married|secondary|     no|      2|    yes| yes|unknown|  5|  may|      76|       1|   -1|       0| unknown| no|        Old|
| 47| blue-collar| married|  unknown|     no|   1506|    yes|  no|unknown|  5|  may|      92|       1|   -1|       0| unknown| no|Middle Aged|